<a href="https://colab.research.google.com/github/alangonzalez91/curso_analisis_datos_1/blob/main/Curso_Analis_Alan_Gonzalez_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# MONTANDO DATOS DEL DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**ETL – Extracción, Transformación y Carga con conexión a Base de Datos remota en Hostinguer + Volcado de datos desde ventas_sistema.csv de un sistema de gestión particular**

*** Los datos csv vinieros extraidos de 2 tablas del sistema de gestión ya Unidos con JOIN.
*** Se adjunta sentencia SQL

```
SELECT
  L.categoria as categoria,
  L.marca as marca,
  V.descripcion as descripcion,
  l.codigo as codigo,
  v.precio_unitario as precio,
  v.cantidad as cantidad,
  v.importe as importe,
  v.fecha as fecha,
  v.hora as hora

FROM ventas_2.detalles as v
left join
  (Select categoria, marca, descripcion, codigo, id
   From productos_2.lista
  ) as L
ON V.producto_nº = L.id

WHERE
  V.producto_nº is not null
```



In [9]:
# Testeo de conexion a base de datos remota en Hostinguer

from sqlalchemy import create_engine, text
import pymysql
from pymysql.constants import CLIENT
from urllib.parse import urlparse, urlunparse
import pandas as pd

# Datos de conexión a base de datos remota en Hostinguer
db_user = 'u599982308_curso_analisis'
db_password = 'Curso_analisis1'
db_host = 'srv655.hstgr.io' # This will be the public IP or domain name of your MySQL server
db_name = 'u599982308_curso_analisis'

# Se crea el SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Se crea la tabla
query = "SELECT 1"

# Se intenta
try:
    df = pd.read_sql(query, engine)
    print("Query exitoro:")
    print(df.head())
except Exception as e:
    print(f"Error: {e}")



Query exitoro:
   1
0  1


In [ ]:
# Creacion de estructura de tabla para volcado de datos
# -----------------------------------------------------
from sqlalchemy import create_engine, text
import pymysql
from pymysql.constants import CLIENT
from urllib.parse import urlparse, urlunparse
import pandas as pd

# Datos de conexión a base de datos remota en Hostinguer
db_user = 'u599982308_curso_analisis'
db_password = 'Curso_analisis1'
db_host = 'srv655.hstgr.io' # This will be the public IP or domain name of your MySQL server
db_name = 'u599982308_curso_analisis'

# Se crea el SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Se crea la tabla
query = "CREATE TABLE ventas (id INT AUTO_INCREMENT PRIMARY KEY, categoria VARCHAR(100), marca VARCHAR(100), descripcion VARCHAR(255), codigo VARCHAR(255), precio FLOAT, cantidad FLOAT,  importe FLOAT, fecha DATE, Hora TIME)";

# Se intenta
try:
    df = pd.read_sql(query, engine)
    print("Query exitoro:")
    print(df.head())
except Exception as e:
    print(f"Error: {e}")


In [21]:
#Se dumpea desde ek csv a la base de datos rempta Hostinguer
# -----------------------------------------------------
from sqlalchemy import create_engine, text
import pymysql
from pymysql.constants import CLIENT
from urllib.parse import urlparse, urlunparse
import pandas as pd

# Datos de conexión a base de datos remota en Hostinguer
db_user = 'u599982308_curso_analisis'
db_password = 'Curso_analisis1'
db_host = 'srv655.hstgr.io' # This will be the public IP or domain name of your MySQL server
db_name = 'u599982308_curso_analisis'

# Se crea el SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Se lee el CSV
df = pd.read_csv('/content/ventas_sistema.csv')

# Se suben los datos
try:
    df.to_sql('ventas', con=engine, if_exists='append', index=False)
    print("Volcado exitoro:")
except Exception as e:
    print(f"Error: {e}")


Volcado exitoro:


In [79]:
# Lectura de datos para consultas
# -----------------------------------------------------
from sqlalchemy import create_engine, text
import pymysql
from pymysql.constants import CLIENT
from urllib.parse import urlparse, urlunparse
import pandas as pd

# Datos de conexión a base de datos remota en Hostinguer
db_user = 'u599982308_curso_analisis'
db_password = 'Curso_analisis1'
db_host = 'srv655.hstgr.io' # This will be the public IP or domain name of your MySQL server
db_name = 'u599982308_curso_analisis'

# Se crea el SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Se crea la tabla
query = "SELECT * From ventas";

# Se intenta
try:
    df = pd.read_sql(query, engine)
    print("Query exitoro:")
except Exception as e:
    print(f"Error: {e}")

# Se aplican correcciones de datos de fecha y hora. Se agrega tabla adicional fecha_hora
pd.options.display.float_format = "{:.2f}".format  # solo 2 decimales
df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d', errors='coerce')
df['Hora'] = pd.to_timedelta(df['Hora'], errors='coerce').apply(lambda x: (pd.Timestamp("1900-01-01") + x).time() if pd.notnull(x) else None)

print(df.head())

Query exitoro:
   id    categoria         marca  \
0   1   FERRETERIA      DURACELL   
1   2  Ferretería       DURACELL   
2   3   FERRETERIA  SOL DE NOCHE   
3   4          LUZ        SV-216   
4   5          LUZ      Lifetime   

                                         descripcion         codigo  precio  \
0                           C2 Duracell Blister de 2    04133300099 4550.00   
1                           D2 Duracell Blister de 2   041333000985 4550.00   
2     Aaa Sol de noche Blister de 4 , Extra duracion  7798087402765 1100.00   
3     Bicicleta SV-216 luz para bicicleta recargable         140001 2600.00   
4  Led bicicleta Lifetime Set de dos luces Led pa...  2400000003212 2200.00   

   cantidad  importe      fecha      Hora  
0      1.00  4550.00 2024-03-07  17:16:06  
1      1.00  4550.00 2024-03-07  17:17:52  
2      1.00  1100.00 2024-03-07  17:36:06  
3      1.00  2600.00 2024-03-07  17:57:54  
4      1.00  2200.00 2024-03-07  18:03:43  


In [81]:
# Descripcion de los datos
# -----------------------------------------------------
#print(df[['fecha','Hora']].head())
print(df.value_counts())

id      categoria    marca              descripcion                                               codigo           precio   cantidad  importe  fecha       Hora    
134193  SANITARIOS   Ginyplas           Canilla Ginyplas CANILLA PVC 1/2                          can0004          1800.00  1.00      1800.00  2025-10-30  06:03:34    1
1       FERRETERIA   DURACELL           C2 Duracell Blister de 2                                  04133300099      4550.00  1.00      4550.00  2024-03-07  17:16:06    1
2       Ferretería   DURACELL           D2 Duracell Blister de 2                                  041333000985     4550.00  1.00      4550.00  2024-03-07  17:17:52    1
3       FERRETERIA   SOL DE NOCHE       Aaa Sol de noche Blister de 4 , Extra duracion            7798087402765    1100.00  1.00      1100.00  2024-03-07  17:36:06    1
4       LUZ          SV-216             Bicicleta SV-216 luz para bicicleta recargable            140001           2600.00  1.00      2600.00  2024-03-07  17:57

**EDA Análisis Exploratorio de Datos**

In [87]:
#conteo
df.value_counts()



,,,,,,,,,,count
id,categoria,marca,descripcion,codigo,precio,cantidad,importe,fecha,Hora,
134193,SANITARIOS,Ginyplas,Canilla Ginyplas CANILLA PVC 1/2,can0004,1800.00,1.00,1800.00,2025-10-30,06:03:34,1
1,FERRETERIA,DURACELL,C2 Duracell Blister de 2,04133300099,4550.00,1.00,4550.00,2024-03-07,17:16:06,1
2,Ferretería,DURACELL,D2 Duracell Blister de 2,041333000985,4550.00,1.00,4550.00,2024-03-07,17:17:52,1
3,FERRETERIA,SOL DE NOCHE,"Aaa Sol de noche Blister de 4 , Extra duracion",7798087402765,1100.00,1.00,1100.00,2024-03-07,17:36:06,1
4,LUZ,SV-216,Bicicleta SV-216 luz para bicicleta recargable,140001,2600.00,1.00,2600.00,2024-03-07,17:57:54,1
...,...,...,...,...,...,...,...,...,...,...
10,Ferretería,Perfecto,Autoperforante Perfecto Punta aguja 8 x 3/4 Phillips,BU0611,20.00,4.00,80.00,2024-03-08,09:06:19,1
9,Ferretería,-,Mecha ezeta Acero rápido 8mm,mec0011,5540.00,1.00,5540.00,2024-03-08,08:35:29,1
8,Ferretería,Apret,Abrazadera Apret 16-10mm,ACA16,740.00,1.00,740.00,2024-03-08,08:31:22,1


In [91]:
#head
df.head()

,id,categoria,marca,descripcion,codigo,precio,cantidad,importe,fecha,Hora
0,1,FERRETERIA,DURACELL,C2 Duracell Blister de 2,04133300099,4550.00,1.00,4550.00,2024-03-07,17:16:06
1,2,Ferretería,DURACELL,D2 Duracell Blister de 2,041333000985,4550.00,1.00,4550.00,2024-03-07,17:17:52
2,3,FERRETERIA,SOL DE NOCHE,"Aaa Sol de noche Blister de 4 , Extra duracion",7798087402765,1100.00,1.00,1100.00,2024-03-07,17:36:06
3,4,LUZ,SV-216,Bicicleta SV-216 luz para bicicleta recargable,140001,2600.00,1.00,2600.00,2024-03-07,17:57:54
4,5,LUZ,Lifetime,Led bicicleta Lifetime Set de dos luces Led pa...,2400000003212,2200.00,1.00,2200.00,2024-03-07,18:03:43


In [92]:
#hdescripcion
df.describe()

,id,precio,cantidad,importe,fecha
count,134193.00,133665.00,134181.00,133719.00,134190
mean,67097.00,3446.87,7.42,5260.92,2024-12-18 19:05:06.639839488
min,1.00,0.00,-500.00,-226000.00,2024-03-07 00:00:00
25%,33549.00,200.00,1.00,600.00,2024-08-10 00:00:00
50%,67097.00,1000.00,1.00,1900.00,2024-12-24 00:00:00
75%,100645.00,3500.00,4.00,4800.00,2025-04-14 00:00:00
max,134193.00,208000.00,1443.00,2597400.00,2025-10-30 00:00:00
std,38738.33,9111.72,29.31,20095.62,NaN


**Preguntas de Negocio**

¿Qué categoria genera más ingresos? = Obtener categorias e importes y ordenar de mayor a menor

¿De la mejor categoría cual fue el año que mejor rindió? = Crear consulta SQL de la categoria y agrupar por año usando sentencia YEAR

¿Cual es el mes con mayor cantidad de ventas de este año 2025? =
Crear consulta SQL de la categoria y agrupar por año usando sentencia CASE


In [96]:
#¿Qué categoria genera más ingresos?

# Lectura de datos para consultas
# -----------------------------------------------------
from sqlalchemy import create_engine, text
import pymysql
from pymysql.constants import CLIENT
from urllib.parse import urlparse, urlunparse
import pandas as pd

# Datos de conexión a base de datos remota en Hostinguer
db_user = 'u599982308_curso_analisis'
db_password = 'Curso_analisis1'
db_host = 'srv655.hstgr.io' # This will be the public IP or domain name of your MySQL server
db_name = 'u599982308_curso_analisis'

# Se crea el SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Se crea la tabla
query = """
      SELECT categoria, sum(importe) as total From ventas
      GROUP BY categoria
      ORDER BY total DESC
    """;

# Se intenta
try:
    df = pd.read_sql(query, engine)
    print("Query exitoro:")
except Exception as e:
    print(f"Error: {e}")

# Se aplican correcciones de datos de fecha y hora. Se agrega tabla adicional fecha_hora
pd.options.display.float_format = "{:.2f}".format  # solo 2 decimales
#df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d', errors='coerce')
#df['Hora'] = pd.to_timedelta(df['Hora'], errors='coerce').apply(lambda x: (pd.Timestamp("1900-01-01") + x).time() if pd.notnull(x) else None)

df.head(20)

Query exitoro:


,categoria,total
0,FERRETERIA,271090587.12
1,SANITARIO,156839972.70
2,ELECTRICIDAD,97781266.80
3,SANITARIOS,35657580.00
4,None,19488172.50
5,PINTURERIA,17393580.00
6,Herrajes,15117180.00
7,Adhesivos,14953800.00
8,BULONERIA,14411061.00
9,SANITARIO,12653607.00


In [101]:
#¿De la mejor categoría cual fue el año que mejor rindió?

# Lectura de datos para consultas
# -----------------------------------------------------
from sqlalchemy import create_engine, text
import pymysql
from pymysql.constants import CLIENT
from urllib.parse import urlparse, urlunparse
import pandas as pd

# Datos de conexión a base de datos remota en Hostinguer
db_user = 'u599982308_curso_analisis'
db_password = 'Curso_analisis1'
db_host = 'srv655.hstgr.io' # This will be the public IP or domain name of your MySQL server
db_name = 'u599982308_curso_analisis'

# Se crea el SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Se crea la tabla
query = """
          SELECT
            YEAR(fecha) as 'año',
            categoria,
            sum(importe) as total,
            count(id) as operaciones
          FROM ventas
          WHERE categoria = 'FERRETERIA'
          GROUP BY
            YEAR(fecha)
        """;

# Se intenta
try:
    df = pd.read_sql(query, engine)
    print("Query exitoro:")
except Exception as e:
    print(f"Error: {e}")

# Se aplican correcciones de datos de fecha y hora. Se agrega tabla adicional fecha_hora
pd.options.display.float_format = "{:.2f}".format  # solo 2 decimales
#df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d', errors='coerce')
#df['Hora'] = pd.to_timedelta(df['Hora'], errors='coerce').apply(lambda x: (pd.Timestamp("1900-01-01") + x).time() if pd.notnull(x) else None)

df.head(20)

Query exitoro:


,año,categoria,total,operaciones
0,2024,FERRETERIA,132272241.78,27882
1,2025,Ferretería,138818345.34,27369


In [102]:
#¿¿Cual es el mes con mayor cantidad de ventas de este año 2025?

# Lectura de datos para consultas
# -----------------------------------------------------
from sqlalchemy import create_engine, text
import pymysql
from pymysql.constants import CLIENT
from urllib.parse import urlparse, urlunparse
import pandas as pd

# Datos de conexión a base de datos remota en Hostinguer
db_user = 'u599982308_curso_analisis'
db_password = 'Curso_analisis1'
db_host = 'srv655.hstgr.io' # This will be the public IP or domain name of your MySQL server
db_name = 'u599982308_curso_analisis'

# Se crea el SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Se crea la tabla
query = """
          SELECT
            CASE MONTH(fecha)
              WHEN 1 THEN "Enero"
              WHEN 2 THEN "Febrero"
              WHEN 3 THEN "Marzo"
              WHEN 4 THEN "Abril"
              WHEN 5 THEN "Mayo"
              WHEN 6 THEN "Junio"
              WHEN 7 THEN "Julio"
              WHEN 8 THEN "Agosto"
              WHEN 9 THEN "Septiembre"
              WHEN 10 THEN "Octubre"
              WHEN 11 THEN "Noviembre"
              WHEN 12 THEN "Diciembre"
              ELSE "Inválido"
              END as 'Mes',
            sum(importe) as total,
            count(id) as operaciones
          FROM ventas
          WHERE fecha >= '2025-01-01'
          GROUP BY
            MONTH(fecha)
        """;

# Se intenta
try:
    df = pd.read_sql(query, engine)
    print("Query exitoro:")
except Exception as e:
    print(f"Error: {e}")

# Se aplican correcciones de datos de fecha y hora. Se agrega tabla adicional fecha_hora
pd.options.display.float_format = "{:.2f}".format  # solo 2 decimales
#df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d', errors='coerce')
#df['Hora'] = pd.to_timedelta(df['Hora'], errors='coerce').apply(lambda x: (pd.Timestamp("1900-01-01") + x).time() if pd.notnull(x) else None)

df.head(20)

Query exitoro:


,Mes,total,operaciones
0,Enero,60247649.10,12882
1,Febrero,44361538.80,8364
2,Marzo,30186280.53,6366
3,Abril,40982787.78,7659
4,Mayo,35674329.00,6360
5,Junio,33804247.50,6264
6,Julio,58023285.18,9942
7,Agosto,38511186.00,6912
8,Septiembre,1705890.15,291
9,Octubre,21600.00,9


**CONCLUCIÓN**

¿Qué categoria genera más ingresos?
    El mes con mejor ingresos fue el de Ferreteria

¿De la mejor categoría cual fue el año que mejor rindió?
    El año de mejor rendimiento fue el 2024

¿Cual es el mes con mayor cantidad de ventas de este año 2025?
    El més con mayores ventas de este año fue Enero